## Fake News Classifier
### LSTM & Bidirectional LSTM


Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [4]:
df=pd.read_csv('train.csv')
df.shape

(20800, 5)

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
###Drop Nan Values
df=df.dropna()
df.shape

(18285, 5)

In [11]:
X=df.drop('label',axis=1) # Input data
X.shape

(18285, 4)

In [12]:
y=df['label'] # Output data
y.shape

(18285,)

In [13]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.14.0'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

from tensorflow.keras.layers import Dropout

In [17]:
voc_size=5000

### Onehot Representation

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manishaa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_repr=[one_hot(words,voc_size) for words in corpus] 
onehot_repr

[[2769, 3659, 1285, 2404, 4132, 469, 1183, 3507, 1995, 4305],
 [3521, 1471, 4005, 3065, 236, 3715, 3783],
 [1984, 2435, 2610, 2185],
 [361, 442, 2409, 4913, 1265, 1845],
 [4193, 236, 4101, 44, 126, 1209, 236, 1375, 2632, 1787],
 [317,
  3642,
  2595,
  1626,
  1773,
  1241,
  738,
  4971,
  2728,
  3418,
  2150,
  4200,
  4565,
  153,
  3783],
 [4905, 245, 2719, 304, 659, 307, 3532, 2837, 2795, 1542, 4377],
 [2403, 1890, 2002, 2467, 928, 2725, 1241, 2512, 2795, 1542, 4377],
 [2937, 1211, 2124, 2856, 1014, 4374, 2630, 1092, 1241, 4731],
 [4959, 1677, 833, 4381, 3768, 1674, 1634, 4883],
 [645, 3995, 4002, 4548, 712, 273, 1614, 452, 677, 2513, 3824],
 [4913, 1342, 4132, 4374, 1241, 928],
 [132, 22, 3665, 4621, 1901, 2788, 706, 4456, 2895],
 [4530, 4102, 1475, 1149, 3866, 4585, 4397, 2795, 1542, 4377],
 [2773, 2796, 3758, 2769, 3003, 2795, 1542, 4377],
 [4661, 2931, 3710, 3878, 3744, 1985, 4064, 626, 2543, 4818],
 [4277, 4621, 1471],
 [2789, 1925, 870, 3885, 1241, 3284, 2383, 3783],
 [2129

### Embedding Representation

In [26]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3507 1995 4305]
 [   0    0    0 ...  236 3715 3783]
 [   0    0    0 ... 2435 2610 2185]
 ...
 [   0    0    0 ... 2795 1542 4377]
 [   0    0    0 ... 3039 1340 1683]
 [   0    0    0 ... 2238 4244 1212]]


In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2769,
       3659, 1285, 2404, 4132,  469, 1183, 3507, 1995, 4305])

## Model

In [28]:
# LSTM
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [29]:
# Bidirectional LSTM
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [30]:
len(embedded_docs),y.shape

(18285, (18285,))

In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [32]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [34]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 45ms/step - loss: 0.3114 - accuracy: 0.8500 - val_loss: 0.2005 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1382 - accuracy: 0.9477 - val_loss: 0.2022 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0901 - accuracy: 0.9677 - val_loss: 0.2108 - val_accuracy: 0.9135
Epoch 4/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0610 - accuracy: 0.9798 - val_loss: 0.3289 - val_accuracy: 0.9147
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0420 - accuracy: 0.9859 - val_loss: 0.3827 - val_accuracy: 0.9137
Epoch 6/10
192/192 [==============================] - 9s 44ms/step - loss: 0.0291 - accuracy: 0.9907 - val_loss: 0.3965 - val_accuracy: 0.9072
Epoch 7/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0174 - accuracy: 0.9949 - val_loss: 0.4895 - val_accuracy: 0.915

### Performance Metrics And Accuracy

In [54]:
y_pred1=(model1.predict(X_test) > 0.5).astype("int64")

189/189 [==============================] - 1s 6ms/step


In [55]:
y_pred1

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int64)

In [37]:
from sklearn.metrics import confusion_matrix

In [56]:
confusion_matrix(y_test,y_pred1)

array([[3127,  292],
       [ 263, 2353]], dtype=int64)

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.908036454018227

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



## Done!